In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn

In [2]:
mnist = input_data.read_data_sets("MNIST.data/", one_hot=True)

#图片尺寸28*28
n_input = 28            #输入一行，一行有28个数据
max_time = 28           #一共28行

lstm_size = 100         #隐层神经单元个数。

n_classes = 10          #10个分类
batch_size = 50         #每批次50个样本
n_batch = mnist.train.num_examples // batch_size


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST.data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST.data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST.data/t10k-images-idx3-ubyte.gz
Extracting MNIST.data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None,10])

#初始化权值
weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes], stddev=0.1))

#初始化偏置值
biases = tf.Variable(tf.constant(0.1,shape=[n_classes]))

#定义RNN网络
def RNN(X, weights,biases):
    #将原本压缩成一行的28*28还原
    inputs = tf.reshape(X, [-1,max_time,n_input])
    
    #定义LSTM基本CELL
    lstm_cell = rnn.BasicLSTMCell(lstm_size)
    
    #final_state[0] 是cell state
    #final_state[1] 是hidden state
    #返回的final_state [0], [1]
    outputs, final_state = tf.nn.dynamic_rnn(lstm_cell, inputs, dtype=tf.float32)  #这是一个计算。传进去两个参数，返回两个参数。
    
    results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    return results

prediction = RNN(x, weights, biases)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))

#使用AdamOptimizer优化器
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_predition = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))
accuracy = tf.reduce_mean(tf.cast(correct_predition, tf.float32))

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x:batch_xs, y:batch_ys})
        acc = sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels})
        print("Iter" + str(epoch) + ", TESTING Accuracy" + str(acc))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Iter0, TESTING Accuracy0.7819
Iter1, TESTING Accuracy0.871
Iter2, TESTING Accuracy0.894
Iter3, TESTING Accuracy0.9014
Iter4, TESTING Accuracy0.914
Iter5, TESTING Accuracy0.9276


笔记：
    ①from tensorflow.contrib import rnn
    
    lstm_cell = rnn.BasicLSTMCell(lstm_size)
    
    定义lstm_cell的函数。原来的已弃用
    
    ②三个计算交叉熵的函数。
    tf.nn.softmax_cross_entropy_with_logits             logits=没有softmax的prediction.逐渐弃用
    tf.nn.sparse_softmax_cross_entropy_with_logits      logits=没有softmax的prediction。计算更快
    tf.nn.softmax_cross_entropy_with_logits_v2          logits=softmax的prediction